In [ ]:
from gensim.test.utils import common_corpus, common_dictionary
from gensim.models.ldamodel import LdaModel
from gensim.models.coherencemodel import CoherenceModel
import numpy as np
import nltk
from nltk import bigrams
from nltk import trigrams
from nltk import ngrams
import itertools
import pandas as pd
import math
import itertools
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
%%capture

!pip install bertopic
!pip install octis
!pip uninstall gensim -y
!pip install git+https://github.com/RaRe-Technologies/gensim.git@refs/pull/3172/merge

from google.colab import drive
drive.mount("/content/drive", force_remount=True)
import os
os.chdir("/content/drive/My Drive/Evaluation/")

from octis.dataset.dataset import Dataset
from data import DataLoader
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import normalize
from sentence_transformers import SentenceTransformer
from octis.evaluation_metrics.diversity_metrics import TopicDiversity
from octis.evaluation_metrics.coherence_metrics import Coherence
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.tokenize import word_tokenize
import string

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
import umap
import hdbscan
from bertopic import BERTopic
import tensorflow_hub

#UN dataset

In [ ]:
%%time
dataloader = DataLoader(dataset="un_dtm").prepare_docs(save="un_dtm.txt").preprocess_octis(output_folder="un_dtm")

created vocab
31744
words filtering done
CPU times: user 2min 5s, sys: 2.78 s, total: 2min 7s
Wall time: 2min 29s


In [ ]:
%%capture

# Prepare data
#dataset, custom = "trump_dtm", True
dataset, custom = "un_dtm", True
data_loader = DataLoader(dataset)
_, timestamps = data_loader.load_docs()
data = data_loader.load_octis(custom)
data = [" ".join(words) for words in data.get_corpus()]

# Match indices
import os
os.listdir(f"./{dataset}")
with open(f"./{dataset}/indexes.txt") as f:
    indices = f.readlines()
    
indices = [int(index.split("\n")[0]) for index in indices]
timestamps = [timestamp for index, timestamp in enumerate(timestamps) if index in indices]
len(data), len(timestamps)

#Trump dataset

In [ ]:
import re
import pandas as pd
from datetime import datetime

# Load data
trump = pd.read_csv('https://drive.google.com/uc?export=download&id=1xRKHaP-QwACMydlDnyFPEaFdtskJuBa6')

# Filter
trump.text = trump.apply(lambda row: re.sub(r"http\S+", "", row.text).lower(), 1)
trump.text = trump.apply(lambda row: " ".join(filter(lambda x:x[0]!="@", row.text.split())), 1)
trump.text = trump.apply(lambda row: " ".join(re.sub("[^a-zA-Z]+", " ", row.text).split()), 1)
trump = trump.loc[(trump.isRetweet == "f") & (trump.text != ""), :]
Timestamps = trump.date.to_list()
timestamps = [Timestamps[i][:4] for i in range(len(Timestamps))]
tweets = trump.text.to_list()
data = tweets

# BERTopic evaluation

To perform Dynamic Topic Modeling with BERTopic we will first need to create a basic topic model using all tweets. The temporal aspect will be ignored as we are, for now, only interested in the topics that reside in those tweets.

In [ ]:
from bertopic import BERTopic
topic_model = BERTopic(min_topic_size=35, verbose=True, top_n_words = 10)
topics, _ = topic_model.fit_transform(tweets)

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/612 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/349 [00:00<?, ?B/s]

Batches:   0%|          | 0/1418 [00:00<?, ?it/s]

2022-08-22 10:17:52,925 - BERTopic - Transformed documents to Embeddings
2022-08-22 10:19:14,746 - BERTopic - Reduced dimensionality
2022-08-22 10:19:22,701 - BERTopic - Clustered reduced embeddings


Now we perform DTM with BERTopic, based on the overall topics detected at the previous step.

In [ ]:
topics_over_time = topic_model.topics_over_time(docs=tweets, 
                                                topics=topics, 
                                                timestamps=timestamps, 
                                                global_tuning=True, 
                                                evolution_tuning=True, 
                                                nr_bins=20, )

13it [00:05,  2.46it/s]


In [ ]:
topics_over_time

,Topic,Words,Frequency,Timestamp
0,-1,"donald, trump, blog, summer, champion",19,2008-12-27 14:48:28.800
1,0,"execute, imagination, donald, pinkett, randal",6,2008-12-27 14:48:28.800
2,2,"begun, schedule, ahead, international, scotland",1,2008-12-27 14:48:28.800
3,3,"cling, wallflower, persona, walls, rather",1,2008-12-27 14:48:28.800
4,4,"habitat, urban, tallest, tall, ranked",1,2008-12-27 14:48:28.800
...,...,...,...,...
1291,72,"josh, true, thanks, agree, so",1,2020-05-26 20:24:00.000
1292,73,"vaccines, inoculate, administered, distributio...",2,2020-05-26 20:24:00.000
1293,82,"great, excellent, okay, good, em",1,2020-05-26 20:24:00.000
1294,97,"transparency, prices, drug, publish, bitter",2,2020-05-26 20:24:00.000


# Evaluation over time 

In [ ]:
####### c-TF-IDF #######
def c_tf_idf(documents, m, ngram_range=(1, 1)):
  count = CountVectorizer(ngram_range=ngram_range, stop_words="english").fit(documents)
  t = count.transform(documents).toarray()
  w = t.sum(axis=1)
  tf = np.divide(t.T, w)
  sum_t = t.sum(axis=0)
  idf = np.log(np.divide(m, sum_t)).reshape(-1, 1)
  tf_idf = np.multiply(tf, idf)
  return tf_idf, count

def extract_top_n_words_per_topic(tf_idf, count, docs_per_topic, n=20):
  '''
  This function allows us to extract the top n words in a topic
  '''
  words = count.get_feature_names()
  labels = list(docs_per_topic.Topic)
  tf_idf_transposed = tf_idf.T
  indices = tf_idf_transposed.argsort()[:, -n:]
  top_n_words = {label: [(words[j], tf_idf_transposed[i][j]) for j in indices[i]][::-1] for i, label in enumerate(labels)}
  return top_n_words

def extract_topic_sizes(df):
  topic_sizes = (df.groupby(['Topic'])
                      .Doc
                      .count()
                      .reset_index()
                      .rename({"Topic": "Topic", "Doc": "Size"}, axis='columns')
                      .sort_values("Size", ascending=False))
  return topic_sizes





####################################################################################
######################### Functions for the evaluation #############################
####################################################################################


#@@@@@@@@@@@@@@@@@@@@@@@@@ Functions for Topic Coherence #@@@@@@@@@@@@@@@@@@@@@@@@@@

def word_proba(data): # data is a list of words
    '''
    This function returns the probability of every word of a list based on their frequency in that list.
    data : is a list of words, rerpresenting the list of words of a text that has been pre-processed.
    '''
    counts = dict()
    words = data
    k = 0

    for word in words:
        if word in counts:
            counts[word] += 1
            k+=1
        else:
            counts[word] = 1
            k+=1

    for word in counts :
      counts[word] = counts[word]/k

    return counts


def N_word_proba(data, N): 
    '''
    Parameters 
    ----------
    data : is a list of words, typically the list of the top-K words of a topic
    N    : is the size of the "window" we consider for quantifying the co-occurrence of words 

    This function returns a dictionnary containing the probability of each "window of words" of a text (represented by data).
    '''
    counts = dict()
    N_grams = list(ngrams(data, N))
    k = 0

    for uplet in N_grams :
        if uplet in counts:
            counts[uplet] += 1
            k+=1
        else:
            counts[uplet] = 1
            k+=1

    for uplet in counts :
      counts[uplet] = counts[uplet]/k

    ###############
    for k in counts :
      permutations = list(itertools.permutations(k))
      permutations.remove(k)
      for permu in permutations :
        if permu in counts :
          counts[k] += counts[permu]

    for k in counts :
      permutations = list(itertools.permutations(k))
      permutations.remove(k)
      for permu in permutations :
        if permu in counts :
          Min = min(counts[k], counts[permu])
          counts[k] = Min
          counts[permu] = Min
    ##############

    return counts



#@@@@@@@@@@@@@@@@@@@@@@@@@ Functions for Topic Diversity #@@@@@@@@@@@@@@@@@@@@@@@@@@

def proportion_unique_words(topics, topk=25):
    """
    This function computes the proportion of unique words, this is how we quantify topic diversity !

    N.B. : the proportion of unique words is a Topic Diversity metric !!!!!!

    Parameters
    ----------
    topics: a list of lists of words
    topk: top k words on which the topic diversity will be computed
    """
    if topk > len(topics[0]):
        raise Exception('Words in topics are less than '+str(topk))
    else:
        unique_words = set()
        for topic in topics:
            unique_words = unique_words.union(set(topic[:topk]))
        puw = len(unique_words) / (topk * len(topics))
        return puw

#Evaluation (BERTopic)

In [ ]:
evaluation = pd.DataFrame(columns = ["timestamp", "TC", "TD"]) #dataframe containing all metrics values for all timestamps

TIMESTAMP = np.unique(topics_over_time["Timestamp"].values)
timestamp = np.unique(timestamps)

for h in range(len(timestamp)) :
  #for t in timestamp : 

  # TWEETS
  dataframe = pd.DataFrame(tweets, columns = ["Doc"])
  dataframe["timestamps"] = timestamps
  DATA = dataframe[dataframe["timestamps"] == timestamp[h]].Doc.values # DATA corresponds to the data from timestamp t

  ########### We concatenate all documents from timestamp[i] into a single document #############

  docs_df = pd.DataFrame(DATA, columns=["Doc"])
  docs_df['timestamp'] = [timestamp[h] for j in range(len(DATA))]
  docs_per_timestamp = docs_df.groupby(['timestamp'], as_index = False).agg({'Doc': ' '.join})
  overall_text = docs_per_timestamp


  ################################################################################
  # We concatenate all documents from timestamp t into a single document we call overall_text

  #overall_text = docs_per_topic.groupby(['overall'], as_index = False).agg({'Doc': ' '.join}) #contains all documents (concatenated) from timestamp t

  # We retrieve the list of words of the overall text (for the computation of npmi)
  text = overall_text.Doc.values[0]
  text_tokens = word_tokenize(text)

  ########## The following line of code takes 30 seconds to execute ! ##########
  list_of_words = [word for word in text_tokens if not word in stopwords.words('english')] #contains the list of words composing the text (without stopwords!)
  ################################################################################

  ####### Topic representation #######
  #### We create the list of topic words for the evaluation

  DFF = topics_over_time[topics_over_time['Timestamp'] == TIMESTAMP[h]]
  if -1 in DFF.Topic.values.tolist() :
    WORDS = DFF.Words.values.tolist()
    WORDS = WORDS[1:]
  else :
    WORDS = DFF.Words.values.tolist()

  topic_words = [WORDS[y].split(", ") for y in range(len(WORDS))]



  topic_npmi = [] #@
  Word_proba = word_proba(list_of_words) # 0 second
    
  ######################################
  ng = N_word_proba(list_of_words, 5)
  #ng = N_word_proba(data, n)
  ng2 = ng.copy()
  n_uplet = dict() # n_uplet contains the real n-uplets of words ! (with no redundant uplets!)

  for N in ng2 :
    permuta = list(itertools.permutations(N))
    permutations = [] #contains combinaisons of N (different than N)
    for i in range(len(permuta)) :
      if permuta[i] != N :
        permutations.append(permuta[i])
    
    p = 0
    for permu in permutations :
      if permu in ng :
        if ng[permu] != "Nan" :
          p += 1

    if p != 0 :
      for permu in permutations :
        ng[permu] = "Nan"

    if ng[N] != "Nan" :
      n_uplet[N] = ng2[N] 
  #################################################

  # We possess n_uplet, which contains all text's n_uplets' probabilities !
  # We use them to compute all npmis at once !

  for listi in topic_words :
    #@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
    k = 0
    NPMI = 0
    dd = []

    for word_i in listi :
      for word_j in listi :
        combined1 = (word_i, word_j)
        combined2 = (word_j, word_i)
        if word_i != word_j :
          if combined2 not in dd :
            #NPMI += npmi(word_i, word_j, list_of_words, 5)

            #####
            # We retrieve P(word1,word2)
            combined_proba = 0 # combined_proba = P(word1,word2)
            for kk in n_uplet :
              if (word_i in kk) & (word_j in kk) :
                combined_proba += n_uplet[kk]

            # We retrieve P(word1) & P(word2)
            w1_proba = 0          # w1_proba = P(word1)
            if word_i in Word_proba:
                w1_proba = Word_proba[word_i]

            w2_proba = 0          # w2_proba = P(word2)
            if word_j in Word_proba:
                w2_proba = Word_proba[word_j]

            ###############  NPMI computation  ################

            if float(combined_proba) == (float(w1_proba * w2_proba)) :
              result = 0 # because PMI would be equal to 1
            else : 
              if float(combined_proba) == 0 : 
                result = -1  # because "combined_proba = P(word1,word2) = 0" means that word1 and word 2 occur separately, never together.

              else :
                ###### We compute NPMI ######
                result = 0
                result = np.log((float(combined_proba))/(float(w1_proba * w2_proba))) # This is equal to PMI
                result = result / (-1.0*np.log(float(combined_proba))) # Normalized PMI

            NPMI += result #####
            k = k + 1
            dd.append(combined1)
            dd.append(combined2)
            #veracity[combined1] = npmi(word_i, word_j, list_of_words, 5)

    NPMI = NPMI / k

    topic_npmi.append(NPMI) #contains the npmi score of all topics !


  average_npmi = 0
  q = 0
  for value in topic_npmi :
    average_npmi += value
    q += 1
  average_npmi = average_npmi / q
  print(f"The overall average npmi value, at timestamp {timestamp[h]}, is ", average_npmi)
  print(f"The topic diversity (TD) value, at timestamp {timestamp[h]}, is ", proportion_unique_words(topic_words, topk = 5))

  # We store the average_npmi value and the puw value in a dataframe
  eve = pd.DataFrame([timestamp[h]], columns = ["timestamp"])
  eve["TC"] = average_npmi
  eve["TD"] = proportion_unique_words(topic_words, topk = 5)
  #eve["TD"] = proportion_unique_words(topic_words_TD, topk = 25)
  evaluation = evaluation.append(eve)
  

  ################################################################################
  ################################################################################


The overall average npmi value, at timestamp 2009, is  0.49429520478344563
The topic diversity (TD) value, at timestamp 2009, is  0.97
The overall average npmi value, at timestamp 2010, is  0.47014466772977026
The topic diversity (TD) value, at timestamp 2010, is  0.9545454545454546
The overall average npmi value, at timestamp 2011, is  0.3408285687201124
The topic diversity (TD) value, at timestamp 2011, is  0.9
The overall average npmi value, at timestamp 2012, is  0.052130998920389714
The topic diversity (TD) value, at timestamp 2012, is  0.9153846153846154
The overall average npmi value, at timestamp 2013, is  0.04984356050939987
The topic diversity (TD) value, at timestamp 2013, is  0.8921985815602836
The overall average npmi value, at timestamp 2014, is  0.010033567453664995
The topic diversity (TD) value, at timestamp 2014, is  0.9112903225806451
The overall average npmi value, at timestamp 2015, is  -0.0014927121439589517
The topic diversity (TD) value, at timestamp 2015, is  0

In [ ]:
evaluation

,timestamp,TC,TD
0,2009,0.494295,0.970000
0,2010,0.470145,0.954545
0,2011,0.340829,0.900000
0,2012,0.052131,0.915385
0,2013,0.049844,0.892199
0,2014,0.010034,0.911290
0,2015,-0.001493,0.866187
0,2016,0.072271,0.876033
0,2017,0.030960,0.926214
0,2018,0.043658,0.900901


In [ ]:
evaluation.TD.values.mean()

0.9098212350796936

#Evaluation LDASeq

For the evaluation of LDA Sequence, we use the code from the paper.

In [ ]:
from evaluation import Trainer

In [ ]:
import os
import pandas as pd
import tqdm
from tqdm import tqdm

# Prepare data
dataset, custom = "un_dtm", True
data_loader = DataLoader(dataset)
_, timestamps = data_loader.load_docs()
data = data_loader.load_octis(custom)
data = [" ".join(words) for words in data.get_corpus()]

# Match indices
os.listdir(f"{dataset}")
with open(f"{dataset}/indexes.txt") as f:
    indices = f.readlines()
    
indices = [int(index.split("\n")[0]) for index in indices]
indices_test = {index: True for index in indices}
timestamps = [timestamp for index, timestamp in tqdm(enumerate(timestamps)) if indices_test.get(index)]
len(data), len(timestamps)

51130it [00:00, 1739377.13it/s]


(50422, 50422)

Then, we simply pass the timestamps and run our the trainer for LDAseq:

In [ ]:
params = {
    "num_topics": [50],
    "nr_bins": 9,
    "random_state": 42
}

trainer = Trainer(dataset=dataset,
                  model_name="LDAseq",
                  params=params,
                  custom_dataset=custom,
                  bt_timestamps=timestamps,
                  topk=5,
                  verbose=True)
results = trainer.train()

50422 50422 [9284 4540 5112 5111 4977 5994 6016 5348 4040]


We remove some information from the results as those are quite big to save:

In [ ]:
results[0]["Params"].keys()
del results[0]["Params"]["corpus"]
del results[0]["Params"]["id2word"]
del results[0]["Params"]["time_slice"]

import json
with open(f"LDAseq_un.json", 'w') as f:
    json.dump(results, f)